In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import os.path
import pandas as pd
import torch
import ttach as tta
from tqdm.auto import tqdm
from models import *
from models_ablations import *
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from sklearn import metrics
from config import *
from util import *
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from thop import profile

In [2]:
def get_model(model_name):
    seed_everything(seed)
    if model_name == 'resnet18':
        return Resnet18(data_dir=data_dir, weighted_sampling=True, batch_size=batch_size, learning_rate=learning_rate)
    elif model_name == 'resnet34':
        return Resnet34(data_dir=data_dir, weighted_sampling=True, batch_size=batch_size, learning_rate=learning_rate)
    elif model_name == 'resnet50': 
        return Resnet50(data_dir=data_dir, weighted_sampling=True, batch_size=batch_size, learning_rate=learning_rate)
    elif model_name == 'resnet101': 
        return Resnet101(data_dir=data_dir, weighted_sampling=True, batch_size=batch_size, learning_rate=learning_rate)
    elif model_name == 'densenet121': 
        return Densenet121(data_dir=data_dir, weighted_sampling=True, batch_size=batch_size, learning_rate=learning_rate)
    elif model_name == 'densenet161': 
        return Densenet161(data_dir=data_dir, weighted_sampling=True, batch_size=batch_size, learning_rate=learning_rate)
    elif model_name == 'efficientnetb1':
        return EfficientNetB1(data_dir=data_dir, weighted_sampling=True, batch_size=batch_size, learning_rate=learning_rate)
    elif model_name == 'efficientnetb3': 
        return EfficientNetB3(data_dir=data_dir, weighted_sampling=True, batch_size=batch_size, learning_rate=learning_rate)

In [3]:
%%time

model_names_list = ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'densenet121', 'densenet161', 'efficientnetb1', 'efficientnetb3']
accuracies = {}
for model_name in tqdm(model_names_list):
    
    torch.cuda.empty_cache()
    seed_everything(seed)
    model = get_model(model_name)
    trainer = pl.Trainer(max_epochs=num_epochs, devices=1, accelerator="gpu", deterministic=True, enable_checkpointing=False, logger=False)
    trainer.fit(model)
    trainer.save_checkpoint("models/"+model_name+".ckpt")
    print("============")
    print(model_name)
    print("============")
    print('Val:')
    result_val = get_dx_predictions(trainer, model, split='val', threshold=0.5)
    threshold = find_optimal_cutoff(result_val['true'], result_val['score'])
    display_scores(result_val)
    print('Test:')
    result_test = get_dx_predictions(trainer, model, split='test', threshold=threshold)
    display_scores(result_test)
    result_val.to_csv('../data/baseline/result_val_'+model_name, index=False)
    result_test.to_csv('../data/baseline/result_test_'+model_name, index=False)
    
    # Calculate FLOPs and parameters
    sample_input = torch.randn(1, 3, 224, 224)
    flops, params = profile(model, inputs=(sample_input,), verbose=False)
    print(f"Number of FLOPs for {model_name}: {flops}")
    print(f"Number of parameters for {model_name}: {params}")
    accuracies[model_name] = {
        'accuracy': metrics.balanced_accuracy_score(result_test['true'], result_test['pred']),
        'flops': flops,
        'parameters': params
    }

  0%|          | 0/8 [00:00<?, ?it/s]

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]

  | Name        | Type        | Params
--------------------------------------------
0 | loss        | BCELoss     | 0     
1 | base_model  | ResNet      | 11.2 M
2 | sigm        | Sigmoid     | 0     
3 | accuracy    | Accuracy    | 0     
4 | auroc       | AUROC       | 0     
5 | sensitivity | Recall      | 0     
6 | specificity | Specificity | 0     
--------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.708    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Predicting: 84it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


=====
AUC: 0.8632617015013245
Balanced Acc: 0.7369222843685606
Sensitivity: 0.5523255813953488
Specificity: 0.9215189873417722
=====

Test:


Predicting: 84it [00:00, ?it/s]

=====
AUC: 0.8145000000000001
Balanced Acc: 0.74
Sensitivity: 0.75
Specificity: 0.73
=====

Number of FLOPs for efficientnetb3: 52023040.0
Number of parameters for efficientnetb3: 88833.0
CPU times: user 33min 9s, sys: 13min 36s, total: 46min 46s
Wall time: 1h 30min 48s


In [17]:
result = pd.DataFrame(accuracies).T.sort_values('accuracy', ascending=False).reset_index().rename({'index': 'backbone'}, axis=1)
result.to_csv('../data/baseline/result.csv', index=False)
result

,backbone,accuracy,flops,parameters
0,resnet50,0.805,4.131697e+09,23510081.0
1,resnet101,0.800,7.864390e+09,42502209.0
2,resnet18,0.785,1.823522e+09,11177025.0
3,densenet121,0.785,2.895984e+09,6954881.0
4,densenet161,0.775,7.843635e+09,26474209.0
5,resnet34,0.765,3.678228e+09,21285185.0
6,efficientnetb3,0.740,5.202304e+07,88833.0
7,efficientnetb1,0.720,3.742758e+07,63329.0


In [18]:
accuracies

{'resnet18': {'accuracy': 0.785,
  'flops': 1823522304.0,
  'parameters': 11177025.0},
 'resnet34': {'accuracy': 0.765,
  'flops': 3678227968.0,
  'parameters': 21285185.0},
 'resnet50': {'accuracy': 0.8049999999999999,
  'flops': 4131696640.0,
  'parameters': 23510081.0},
 'resnet101': {'accuracy': 0.8,
  'flops': 7864389632.0,
  'parameters': 42502209.0},
 'densenet121': {'accuracy': 0.785,
  'flops': 2895984128.0,
  'parameters': 6954881.0},
 'densenet161': {'accuracy': 0.775,
  'flops': 7843635264.0,
  'parameters': 26474209.0},
 'efficientnetb1': {'accuracy': 0.72,
  'flops': 37427584.0,
  'parameters': 63329.0},
 'efficientnetb3': {'accuracy': 0.74,
  'flops': 52023040.0,
  'parameters': 88833.0}}